In [29]:
from numpy import *
def loadDataSet(filename):
    dataMat=[]
    fr=open(filename)
    for line in fr.readlines():
        curline=line.strip().split('\t')
        floatLine=list(map(float,curline))#把每行映射成为浮点数
        dataMat.append(floatLine)
    return dataMat

In [21]:
def binSplitDataSet(dataSet,feature,value):
    #根据feature对数据集进行划分，对于数据每一行，根据feature取值进行划分（取列方向的满足条件值作为行下标）
    index_gt=where(dataSet[:,feature]>value)[0]#取axis=0，列方向的值
    index_lt=where(dataSet[:,feature]<=value)[0]
    #map0=dataSet[ nonzero(dataSet[:,feature]>value)[0],: ]
    #map1=dataSet[ nonzero(dataSet[:,feature]<=value)[0],:]
    map0=dataSet[index_gt,:]
    map1=dataSet[index_lt,:]
    return map0,map1

In [27]:
def createTree(dataSet,leafType=regLeaf,errType=regErr,ops=(1,4)):
    feat,val=chooseBestSplit(dataSet,leafType,errType,ops)
    #满足条件，递归返回
    if feat==None:return val
    retTree={}
    retTree['spInd']=feat
    retTree['spval']=val
    lSet,rSet=binSplitDataSet(dataSet,feat,val)
    retTree['left']=createTree(lSet,leafType,errType,ops)
    retTree['right']=createTree(rSet,leafType,errType,ops)
    return retTree

In [23]:
testMat=mat(eye(4))
mat0,mat1=binSplitDataSet(testMat,1,0.5)
mat0

matrix([[ 0.,  1.,  0.,  0.]])

In [24]:
def regLeaf(dataSet):
    return mean(dataSet[:,-1])
def regErr(dataSet):
    return var(dataSet[:,-1])*shape(dataSet)[0]

In [44]:
def chooseBestSplit(dataSet,leafType=regLeaf,errType=regErr,ops=(1,4)):
    tolS=ops[0];tolN=ops[1]
    if len(set(dataSet[:,-1].T.tolist()[0]))==1:
        return None,leafType(dataSet)
    m,n=shape(dataSet)
    S=errType(dataSet)
    bestS=inf;bestIndex=0;bestValue=0
    for featIndex in range(0,n-1):
        #print(dataSet[:,featIndex])
        uniValList=dataSet[:,featIndex].flatten().tolist()[0]
        uniValSet=set(uniValList)
        for splitVal in uniValSet:
            mat0,mat1=binSplitDataSet(dataSet,featIndex,splitVal)
            if shape(mat0)[0]<tolN or shape(mat1)[0]<tolN:
                continue
            newS=errType(mat0)+errType(mat1)
            if newS<bestS:
                bestIndex=featIndex
                bestValue=splitVal
                bestS=newS
    if S-bestS<tolS:
        return None,leafType(dataSet)
    mat0,mat1=binSplitDataSet(dataSet,bestIndex,bestValue)
    
    if shape(mat0)[0]<tolN or shape(mat1)[0]<tolN:
        return None,leafType(dataSet)
    return bestIndex,bestValue

In [46]:
myDat=loadDataSet('ex0.txt')
myDat=mat(myDat)
createTree(myDat)

{'left': {'left': {'left': 3.9871631999999999,
   'right': 2.9836209534883724,
   'spInd': 1,
   'spval': 0.797583},
  'right': 1.980035071428571,
  'spInd': 1,
  'spval': 0.582002},
 'right': {'left': 1.0289583666666666,
  'right': -0.023838155555555553,
  'spInd': 1,
  'spval': 0.197834},
 'spInd': 1,
 'spval': 0.39435}